In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun  2 04:38:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive #Only if you are using Google Drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive
Mounted at /content/gdrive
/content/gdrive/My Drive


In [ ]:
cd /content/gdrive/MyDrive/content/img_dir

/content/gdrive/MyDrive/content/img_dir


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

from keras import applications
from keras.models import Model
import keras 
from keras.applications.vgg16 import VGG16 #Importing the VGG16 Model
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D,BatchNormalization
#from keras.applications import MobileNetV2
#from keras.optimizers import SGD
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input  

In [ ]:
training_dir = '/content/gdrive/MyDrive/content/img_dir/train'
validation_dir = '/content/gdrive/MyDrive/content/img_dir/test'
test_dir = '/content/gdrive/MyDrive/content/img_dir/test'

folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))
print('Total training classes '+str(len(glob(training_dir + '/*'))))
print('Total test classes '+str(len(glob(test_dir + '/*'))))
print('Toral validation classes '+str(len(glob(validation_dir + '/*'))))

Total Classes = 10
Total training classes 10
Total test classes 10
Toral validation classes 10


Tensorflow variant of VGG16

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
#BATCH_SIZE = 255
IMG_SIZE = (224, 224)

#train_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/train'
#validation_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/test/'
data_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/all_labeled/'

vgg16 = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet')


feature_extractor_model = vgg16

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

class_names = np.array(train_ds.class_names)
print(class_names)

Found 49449 files belonging to 217 classes.
Using 39560 files for training.
Found 49449 files belonging to 217 classes.
Using 9889 files for validation.
['Accelerating_and_revving_and_vroom' 'Accordion' 'Acoustic_guitar'
 'Aircraft' 'Alarm' 'Animal' 'Applause' 'Bark' 'Bass_drum' 'Bass_guitar'
 'Bathtub_filling_or_washing' 'Bell' 'Bicycle' 'Bicycle_bell' 'Bird'
 'Bird_vocalization_and_bird_call_and_bird_song' 'Boat_and_Water_vehicle'
 'Boiling' 'Boom' 'Bowed_string_instrument' 'Brass_instrument' 'Breathing'
 'Burping_and_eructation' 'Bus' 'Buzz' 'Camera' 'Car' 'Car_passing_by'
 'Cat' 'Chatter' 'Cheering' 'Chewing_and_mastication'
 'Chicken_and_rooster' 'Child_speech_and_kid_speaking' 'Chime'
 'Chink_and_clink' 'Chirp_and_tweet' 'Chuckle_and_chortle' 'Church_bell'
 'Clapping' 'Clock' 'Coin_dropping' 'Computer_keyboard' 'Conversation'
 'Cough' 'Cowbell' 'Crack' 'Crackle' 'Crash_cymbal' 'Cricket' 'Crow'
 'Crowd' 'Crumpling_and_crinkling' 'Crushing' 'Crying_and_sobbing'
 'Cupboard_open_or_c

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 224, 224, 3)
(32,)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pylab as plabt
import numpy as np
import os
import PIL
import time
import datetime
import PIL.Image as Image

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

(32, 7, 7, 512)


In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 7, 7, 512)         14714688  
                                                                 
 dense (Dense)               (None, 7, 7, 217)         111321    
                                                                 
Total params: 14,826,009
Trainable params: 111,321
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-3),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

#log = "/content/gdrive/MyDrive/content/img_dir/logs/vgg16/all_combined/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log, histogram_freq=1) # Enable histogram computation for every epoch.

In [ ]:
NUM_EPOCHS = 40

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS)

Epoch 1/40


InvalidArgumentError: ignored

In [ ]:
model.save('/content/gdrive/MyDrive/content/vgg16_all_combined_headless_final'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
model.save('/content/gdrive/MyDrive/content/vgg16_all_combined_headless_final.'+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")+'.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
 %reload_ext tensorboard

In [ ]:
%tensorboard --logdir="/content/gdrive/MyDrive/content/img_dir/logs/vgg16/all_combined/fit/"

In [ ]:
tf.keras.applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)